In [1]:
import pandas as pd

# Load the dataset
file_path = '/content/Police_Department_Incident_Reports__2018_to_Present_20250606.csv'
df = pd.read_csv(file_path)

In [2]:
# Preview data
print("🔹 Dataset Shape:", df.shape)

🔹 Dataset Shape: (213977, 35)


In [3]:
print("🔹 Column Names:\n", df.columns)

🔹 Column Names:
 Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident ID',
       'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District',
       'Supervisor District 2012', 'Latitude', 'Longitude', 'Point',
       'Neighborhoods', 'ESNCAG - Boundary File',
       'Central Market/Tenderloin Boundary Polygon - Updated',
       'Civic Center Harm Reduction Project Boundary',
       'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas',
       'Current Supervisor Districts', 'Current Police Districts'],
      dtype='object')


In [4]:
df.head(5)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
0,2023/08/06 11:39:00 AM,2023/08/06,11:39,2023,Sunday,2023/08/06 11:40:00 AM,130460971012,1304609,230519316,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019/06/10 10:23:00 AM,2019/06/10,10:23,2019,Monday,2019/06/26 09:00:00 AM,81573309024,815733,190451474,191731989.0,...,-122.483025,POINT (-122.48302459716797 37.72120666503906),42.0,NaN,NaN,NaN,NaN,NaN,8.0,10.0
2,2023/12/03 03:53:00 AM,2023/12/03,03:53,2023,Sunday,2023/12/03 03:58:00 AM,134286907041,1342869,230803880,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024/02/04 03:00:00 AM,2024/02/04,03:00,2024,Sunday,2024/02/04 03:00:00 AM,136167507041,1361675,240075762,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/11/05 12:00:00 AM,2023/11/05,00:00,2023,Sunday,2023/11/05 12:08:00 AM,133474464085,1334744,230789199,233090002.0,...,-122.446910,POINT (-122.44691011930168 37.80328399631487),17.0,NaN,NaN,NaN,NaN,NaN,6.0,4.0


In [5]:
# Step 1: Parse the 'Incident Datetime' field
df['Incident Datetime'] = pd.to_datetime(df['Incident Datetime'], errors='coerce')

In [6]:
# Step 2: Extract datetime features
df['Hour'] = df['Incident Datetime'].dt.hour
df['Minute'] = df['Incident Datetime'].dt.minute
df['Day'] = df['Incident Datetime'].dt.day
df['Month'] = df['Incident Datetime'].dt.month_name()
df['Year'] = df['Incident Datetime'].dt.year
df['YearMonth'] = df['Incident Datetime'].dt.to_period('M').astype(str)

In [7]:
# Step 3: Define time of day categories
def assign_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

In [8]:
df['TimeOfDay'] = df['Hour'].apply(assign_time_of_day)

# Step 4: Flag for evening crimes
df['IsEveningCrime'] = df['TimeOfDay'].apply(lambda x: 1 if x == 'Evening' else 0)

# Step 5: Check transformed columns
df[['Incident Datetime', 'Hour', 'TimeOfDay', 'IsEveningCrime', 'YearMonth']].head()


,Incident Datetime,Hour,TimeOfDay,IsEveningCrime,YearMonth
0,2023-08-06 11:39:00,11,Morning,0,2023-08
1,2019-06-10 10:23:00,10,Morning,0,2019-06
2,2023-12-03 03:53:00,3,Night,0,2023-12
3,2024-02-04 03:00:00,3,Night,0,2024-02
4,2023-11-05 00:00:00,0,Night,0,2023-11


In [9]:
final_columns = [
    'Incident Datetime', 'Incident Year', 'Incident Day of Week', 'Hour',
    'TimeOfDay', 'IsEveningCrime', 'Incident Category', 'Incident Subcategory',
    'Incident Description', 'Resolution', 'Police District',
    'Analysis Neighborhood', 'Latitude', 'Longitude', 'YearMonth'
]

In [ ]:
# Remove rows with invalid hours
df = df[(df['Hour'] >= 0) & (df['Hour'] <= 23)]

In [10]:
df_final = df[final_columns].dropna(subset=['Incident Datetime', 'Incident Category', 'Police District'])

In [11]:
df_final.head()

,Incident Datetime,Incident Year,Incident Day of Week,Hour,TimeOfDay,IsEveningCrime,Incident Category,Incident Subcategory,Incident Description,Resolution,Police District,Analysis Neighborhood,Latitude,Longitude,YearMonth
0,2023-08-06 11:39:00,2023,Sunday,11,Morning,0,Other Offenses,Other Offenses,"License Plate, Recovered",Open or Active,Out of SF,NaN,NaN,NaN,2023-08
1,2019-06-10 10:23:00,2019,Monday,10,Morning,0,Fraud,Fraud,"Fraudulent Game or Trick, Obtaining Money or P...",Unfounded,Taraval,Lakeshore,37.721207,-122.483025,2019-06
2,2023-12-03 03:53:00,2023,Sunday,3,Night,0,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Out of SF,NaN,NaN,NaN,2023-12
3,2024-02-04 03:00:00,2024,Sunday,3,Night,0,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Out of SF,NaN,NaN,NaN,2024-02
4,2023-11-05 00:00:00,2023,Sunday,0,Night,0,Other Miscellaneous,Other,Investigative Detention,Open or Active,Northern,Marina,37.803284,-122.446910,2023-11


In [12]:
# Export cleaned data
df_final.to_csv('sf_crime_data_cleaned.csv', index=False)
print("Cleaned data exported: 'sf_crime_data_cleaned.csv'")

✅ Cleaned data exported: 'sf_crime_data_cleaned.csv'
